## prepare data from Lisa dataset

This notebook generates xml files and a data pickle file.
File location:
.
|-img/stop_sign1.jpg
|     stop_sign2.jpg
|     .
|     .
|     .
|
|-xml/ (blank file)
|
|-annotations.csv


annotations.csv should have the following structure, derived from the original Lisa dataset CSV format:
```
Filename;Annotation tag;Upper left corner X;Upper left corner Y;Lower right corner X;Lower right corner Y;Occluded,On another road;Origin file;Origin frame number;Origin track;Origin track frame number
```

In [1]:
# Import necessary modules

import pickle
import os
import subprocess
import pandas as pd
import numpy as np
from PIL import Image
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [2]:
# set constants
jpegimagepath = './img/'
xmlfile_destination = './xml/'

# parse information from csv data
df1 = pd.read_csv('annotations.csv')
df1 = df1[df1['Annotation tag'] == 'stop']

df1 = df1.drop(['Unnamed: 0',
              'Occluded,On another road',
              'Origin file',
              'Origin frame number',
              'Origin track frame number',
              'Origin track'],
             axis=1)

df1 = df1.rename(index=str,
               columns={'Filename': 'filename',
                        'Annotation tag': 'object',
                        'Upper left corner X': 'xmin',
                        'Upper left corner Y': 'ymin',
                        'Lower right corner X': 'xmax',
                        'Lower right corner Y': 'ymax'})

df1 = df1.replace({'png': 'jpg'}, regex=True)


# parse information from image data
imgfile_names = os.listdir(jpegimagepath)
annotation_list = []
imagesize_list = []

subprocess.call(['cd img/ && mogrify -format jpg *.png && rm *.png && ls -a && cd ..'], shell=True)

for name in imgfile_names:
    im = Image.open(jpegimagepath + name)
    t = list(im.size)
    t.insert(0,name)
    imagesize_list.append(t)
    
df2 = pd.DataFrame(imagesize_list).rename(index=str, columns={0: 'filename',
                                                              1: 'width',
                                                              2: 'height'})


# concatenate two dataframes together
df = pd.merge(df1, df2)
df['coord'] = df[['xmin','ymin','xmax','ymax']].values.tolist()
df = df.groupby(['filename','object','height','width'])['coord'].apply(list)
df = pd.DataFrame(df).reset_index()
df['d'] = df.apply(lambda row: {row['object']:row['coord']}, axis=1)
df = df.groupby(['filename','height','width'])['d'].apply(list)

df

filename                         height  width
stop_1330545910.avi_image0.jpg   522     1024                    [{'stop': [[862, 104, 916, 158]]}]
stop_1330545910.avi_image1.jpg   522     1024                     [{'stop': [[922, 88, 982, 148]]}]
stop_1330545935.avi_image0.jpg   522     1024     [{'stop': [[376, 138, 413, 175], [709, 203, 72...
stop_1330545935.avi_image1.jpg   522     1024     [{'stop': [[412, 145, 451, 185], [759, 215, 77...
stop_1330545935.avi_image10.jpg  522     1024                     [{'stop': [[903, 81, 976, 155]]}]
stop_1330545935.avi_image2.jpg   522     1024     [{'stop': [[450, 145, 490, 185], [810, 218, 82...
stop_1330545935.avi_image3.jpg   522     1024     [{'stop': [[487, 132, 530, 176], [862, 212, 88...
stop_1330545935.avi_image4.jpg   522     1024     [{'stop': [[531, 128, 577, 174], [921, 214, 94...
stop_1330545935.avi_image5.jpg   522     1024     [{'stop': [[579, 128, 628, 178], [985, 218, 10...
stop_1330545935.avi_image6.jpg   522     1024        

In [3]:
# generate XML files

x = 0
for row in df:
    annotation = ET.Element('annotation')
    filename = ET.Element('filename')
    size = ET.Element('size')
    height = ET.SubElement(size, 'height')
    width = ET.SubElement(size, 'width')
    depth = ET.SubElement(size, 'depth')
    
    filename.text = df.index[x][0]
    height.text = str(df.index[x][1])
    width.text = str(df.index[x][2])
    annotation.extend((filename, size))
    
    for i in range(len(df[x][0]['stop'])):
        obj = ET.SubElement(annotation, 'object')
        bndbox = ET.SubElement(obj, 'bndbox')
        name = ET.SubElement(obj, 'name')
        xmin = ET.SubElement(bndbox, 'xmin')
        ymin = ET.SubElement(bndbox, 'ymin')
        xmax = ET.SubElement(bndbox, 'xmax')
        ymax = ET.SubElement(bndbox, 'ymax')
        
        name.text = 'stop'
        xmin.text = str(df[x][0][name.text][i][0])
        ymin.text = str(df[x][0][name.text][i][1])
        xmax.text = str(df[x][0][name.text][i][2])
        ymax.text = str(df[x][0][name.text][i][3])
        
    tree = ET.ElementTree(annotation)
    ET.dump(tree)
    tree.write(xmlfile_destination + '.'.join(filename.text.split('.')[:-1]) + '.xml')
    x+=1

<annotation><filename>stop_1330545910.avi_image0.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>862</xmin><ymin>104</ymin><xmax>916</xmax><ymax>158</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1330545910.avi_image1.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>922</xmin><ymin>88</ymin><xmax>982</xmax><ymax>148</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1330545935.avi_image0.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>376</xmin><ymin>138</ymin><xmax>413</xmax><ymax>175</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>709</xmin><ymin>203</ymin><xmax>726</xmax><ymax>227</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1330545935.avi_image1.jpg</filename><size><height>522</height><width>1024</width><depth /></size><ob

<annotation><filename>stop_1330546190.avi_image12.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>805</xmin><ymin>228</ymin><xmax>829</xmax><ymax>254</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>357</xmin><ymin>231</ymin><xmax>381</xmax><ymax>255</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1330546190.avi_image13.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>814</xmin><ymin>227</ymin><xmax>838</xmax><ymax>253</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>350</xmin><ymin>229</ymin><xmax>375</xmax><ymax>254</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1330546190.avi_image14.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>822</xmin><ymin>204</ymin><xmax>847</xmax><ymax>230</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>

<annotation><filename>stop_1331866061.avi_image27.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>865</xmin><ymin>77</ymin><xmax>931</xmax><ymax>144</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331866061.avi_image28.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>904</xmin><ymin>60</ymin><xmax>978</xmax><ymax>135</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331866061.avi_image3.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>612</xmin><ymin>179</ymin><xmax>628</xmax><ymax>196</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>291</xmin><ymin>179</ymin><xmax>308</xmax><ymax>196</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331866061.avi_image4.jpg</filename><size><height>522</height><width>1024</width><depth /></size><o

<annotation><filename>stop_1331866288.avi_image26.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>844</xmin><ymin>102</ymin><xmax>902</xmax><ymax>160</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331866288.avi_image27.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>873</xmin><ymin>92</ymin><xmax>935</xmax><ymax>155</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331866288.avi_image28.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>906</xmin><ymin>80</ymin><xmax>973</xmax><ymax>148</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331866288.avi_image29.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>943</xmin><ymin>61</ymin><xmax>1016</xmax><ymax>134</ymax></bndbox><name>stop</name></object>

<annotation><filename>stop_1331867195.avi_image26.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>883</xmin><ymin>131</ymin><xmax>912</xmax><ymax>179</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331867195.avi_image27.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>898</xmin><ymin>129</ymin><xmax>928</xmax><ymax>180</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331867195.avi_image28.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>918</xmin><ymin>126</ymin><xmax>948</xmax><ymax>180</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1331867195.avi_image29.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>942</xmin><ymin>122</ymin><xmax>973</xmax><ymax>178</ymax></bndbox><name>stop</name></objec

...

<annotation><filename>stop_1333392951.avi_image0.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>921</xmin><ymin>85</ymin><xmax>975</xmax><ymax>143</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333392951.avi_image1.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>948</xmin><ymin>76</ymin><xmax>1004</xmax><ymax>138</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333393835.avi_image0.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>794</xmin><ymin>187</ymin><xmax>823</xmax><ymax>217</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333393835.avi_image1.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>854</xmin><ymin>187</ymin><xmax>883</xmax><ymax>217</ymax></bndbox><name>stop</name></object></a

<annotation><filename>stop_1333396743.avi_image18.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>737</xmin><ymin>157</ymin><xmax>774</xmax><ymax>194</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333396743.avi_image19.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>751</xmin><ymin>151</ymin><xmax>789</xmax><ymax>189</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333396743.avi_image2.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>652</xmin><ymin>191</ymin><xmax>672</xmax><ymax>210</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333396743.avi_image20.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>761</xmin><ymin>144</ymin><xmax>801</xmax><ymax>185</ymax></bndbox><name>stop</name></object

<annotation><filename>stop_1333397461.avi_image4.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>197</xmin><ymin>164</ymin><xmax>218</xmax><ymax>186</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>512</xmin><ymin>166</ymin><xmax>529</xmax><ymax>187</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397461.avi_image5.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>186</xmin><ymin>163</ymin><xmax>207</xmax><ymax>185</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>511</xmin><ymin>164</ymin><xmax>528</xmax><ymax>185</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397461.avi_image6.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>175</xmin><ymin>160</ymin><xmax>197</xmax><ymax>183</ymax></bndbox><name>stop</name></object><object><bndbox><xmin>513

<annotation><filename>stop_1333397574.avi_image3.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>689</xmin><ymin>189</ymin><xmax>709</xmax><ymax>209</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397574.avi_image4.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>698</xmin><ymin>189</ymin><xmax>718</xmax><ymax>209</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397574.avi_image5.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>707</xmin><ymin>195</ymin><xmax>728</xmax><ymax>216</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397574.avi_image6.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>717</xmin><ymin>185</ymin><xmax>739</xmax><ymax>207</ymax></bndbox><name>stop</name></object></

<annotation><filename>stop_1333397736.avi_image4.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>734</xmin><ymin>181</ymin><xmax>765</xmax><ymax>212</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397736.avi_image5.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>742</xmin><ymin>165</ymin><xmax>774</xmax><ymax>198</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397736.avi_image6.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>752</xmin><ymin>150</ymin><xmax>785</xmax><ymax>184</ymax></bndbox><name>stop</name></object></annotation>
<annotation><filename>stop_1333397736.avi_image7.jpg</filename><size><height>522</height><width>1024</width><depth /></size><object><bndbox><xmin>768</xmin><ymin>142</ymin><xmax>802</xmax><ymax>178</ymax></bndbox><name>stop</name></object></

In [4]:
# extract data from xml and create pickle data

class XML_preprocessor(object):

    def __init__(self, data_path):
        self.path_prefix = data_path
        self.num_classes = 1
        self.data = dict()
        self._preprocess_XML()

    def _preprocess_XML(self):
        filenames = os.listdir(self.path_prefix)
        for filename in filenames:
            tree = ET.parse(self.path_prefix + filename)
            root = tree.getroot()
            bounding_boxes = []
            one_hot_classes = []
            size_tree = root.find('size')
            width = float(size_tree.find('width').text)
            height = float(size_tree.find('height').text)
            for object_tree in root.findall('object'):
                for bounding_box in object_tree.iter('bndbox'):
                    xmin = float(bounding_box.find('xmin').text)/width
                    ymin = float(bounding_box.find('ymin').text)/height
                    xmax = float(bounding_box.find('xmax').text)/width
                    ymax = float(bounding_box.find('ymax').text)/height
                bounding_box = [xmin,ymin,xmax,ymax]
                bounding_boxes.append(bounding_box)
                class_name = object_tree.find('name').text
                one_hot_class = self._to_one_hot(class_name)
                one_hot_classes.append(one_hot_class)
            image_name = root.find('filename').text
            bounding_boxes = np.asarray(bounding_boxes)
            one_hot_classes = np.asarray(one_hot_classes)
            image_data = np.hstack((bounding_boxes, one_hot_classes))
            self.data[image_name] = image_data

    def _to_one_hot(self,name):
        one_hot_vector = [0] * self.num_classes
        if name == 'stop':
            one_hot_vector[0] = 1
            """
        elif name == 'bicycle':
            one_hot_vector[1] = 1
        elif name == 'bird':
            one_hot_vector[2] = 1
        elif name == 'boat':
            one_hot_vector[3] = 1
        elif name == 'bottle':
            one_hot_vector[4] = 1
        elif name == 'bus':
            one_hot_vector[5] = 1
        elif name == 'car':
            one_hot_vector[6] = 1
        elif name == 'cat':
            one_hot_vector[7] = 1
        elif name == 'chair':
            one_hot_vector[8] = 1
        elif name == 'cow':
            one_hot_vector[9] = 1
        elif name == 'diningtable':
            one_hot_vector[10] = 1
        elif name == 'dog':
            one_hot_vector[11] = 1
        elif name == 'horse':
            one_hot_vector[12] = 1
        elif name == 'motorbike':
            one_hot_vector[13] = 1
        elif name == 'person':
            one_hot_vector[14] = 1
        elif name == 'pottedplant':
            one_hot_vector[15] = 1
        elif name == 'sheep':
            one_hot_vector[16] = 1
        elif name == 'sofa':
            one_hot_vector[17] = 1
        elif name == 'train':
            one_hot_vector[18] = 1
        elif name == 'tvmonitor':
            one_hot_vector[19] = 1
            """
        else:
            print('unknown label: %s' %name)
        return one_hot_vector
    
data = XML_preprocessor(xmlfile_destination).data
pickle.dump(data,open('data.p','wb'))